# Import Libraries

In [1]:
import numpy as np
# Torch
import torch
from torchvision import datasets, transforms
import torch.utils.data as Data
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
# Visualization
from matplotlib import pyplot as plt

In [2]:
# Dataloader
def read_bci_data():
    S4b_train = np.load('S4b_train.npz')
    X11b_train = np.load('X11b_train.npz')
    S4b_test = np.load('S4b_test.npz')
    X11b_test = np.load('X11b_test.npz')

    train_data = np.concatenate((S4b_train['signal'], X11b_train['signal']), axis=0)
    train_label = np.concatenate((S4b_train['label'], X11b_train['label']), axis=0)
    test_data = np.concatenate((S4b_test['signal'], X11b_test['signal']), axis=0)
    test_label = np.concatenate((S4b_test['label'], X11b_test['label']), axis=0)

    train_label = train_label - 1
    test_label = test_label -1
    train_data = np.transpose(np.expand_dims(train_data, axis=1), (0, 1, 3, 2))
    test_data = np.transpose(np.expand_dims(test_data, axis=1), (0, 1, 3, 2))

    mask = np.where(np.isnan(train_data))
    train_data[mask] = np.nanmean(train_data)

    mask = np.where(np.isnan(test_data))
    test_data[mask] = np.nanmean(test_data)

    print(train_data.shape, train_label.shape, test_data.shape, test_label.shape)

    return train_data, train_label, test_data, test_label

In [31]:
# Hyper parameters
batch_size = 64
num_workers = 0
epoch = 300
lr = 1e-2
device = 'cuda'

In [43]:
train_data, train_label, test_data, test_label = read_bci_data()
train_data = torch.from_numpy(train_data).to(device=device, dtype=torch.float)
train_label = torch.from_numpy(train_label).to(device=device, dtype=torch.float)
test_data = torch.from_numpy(test_data).to(device=device, dtype=torch.float)
test_label = torch.from_numpy(test_label).to(device=device, dtype=torch.float)

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [44]:
train_tensor = Data.TensorDataset(train_data, train_label)
test_tensor = Data.TensorDataset(test_data, test_label)

train_loader = Data.DataLoader(
    dataset = train_tensor,
    batch_size = batch_size,
    shuffle = True,
    num_workers = num_workers
)

test_loader = Data.DataLoader(
    dataset = test_tensor,
    batch_size = batch_size,
    shuffle = True,
    num_workers = num_workers
)

In [101]:
class EEGNet(torch.nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.firstConv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False),
            nn.BatchNorm2d(16, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True)
        )
        self.depthwiseConv = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False),
            nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True),
            nn.ELU(alpha=1.0),
            nn.AvgPool2d(kernel_size=(1,4), stride=(1, 4), padding=0),
            nn.Dropout(p=0.25)
        )
        self.separableConv = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False),
            nn.BatchNorm2d(32, eps=1e-5, momentum=0.1, affine=True, track_running_stats=True),
            nn.ELU(alpha=1.0),
            nn.AvgPool2d(kernel_size=(1,8), stride=(1, 8), padding=0),
            nn.Dropout(p=0.25)
        )
        self.classify = nn.Sequential(
            nn.Linear(in_features=736, out_features=2, bias=True)
        )
    def forward(self, x):
        res = self.firstConv(x)
        res = self.depthwiseConv(res)
        res = self.separableConv(res)
        res = res.view(res.size(0), -1)
        res = self.classify(res)
        return res

In [102]:
model = EEGNet()
model.to(device)

EEGNet(
  (firstConv): Sequential(
    (0): Conv2d(1, 16, kernel_size=(1, 51), stride=(1, 1), padding=(0, 25), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (depthwiseConv): Sequential(
    (0): Conv2d(16, 32, kernel_size=(2, 1), stride=(1, 1), groups=16, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0)
    (4): Dropout(p=0.25)
  )
  (separableConv): Sequential(
    (0): Conv2d(32, 32, kernel_size=(1, 15), stride=(1, 1), padding=(0, 7), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0)
    (4): Dropout(p=0.25)
  )
  (classify): Sequential(
    (0): Linear(in_features=736, out_features=2, bias=True)
  )
)

In [103]:
# Define optimizer and loss function
Loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [104]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        target = target.long()
        
        optimizer.zero_grad()
        out = model(data)
        loss = Loss(out, target)
        loss.backward()
        optimizer.step()
        print('Train epoch: {} [{}/{} ({:.0f})%]\t Loss: {:.6f}'.format(epoch,
                                                                     batch_idx * len(data),
                                                                     len(train_loader.dataset),
                                                                     100. * batch_idx / len(train_loader),
                                                                     loss.item()))
        
        

In [105]:
for i in range(1, epoch+1):
    train(model, i)

Train epoch: 1 [0/1080 (0)%]	 Loss: 0.718940
Train epoch: 1 [64/1080 (6)%]	 Loss: 0.594982
Train epoch: 1 [128/1080 (12)%]	 Loss: 0.698169
Train epoch: 1 [192/1080 (18)%]	 Loss: 0.758618
Train epoch: 1 [256/1080 (24)%]	 Loss: 0.614534
Train epoch: 1 [320/1080 (29)%]	 Loss: 0.501216
Train epoch: 1 [384/1080 (35)%]	 Loss: 0.575445
Train epoch: 1 [448/1080 (41)%]	 Loss: 0.630576
Train epoch: 1 [512/1080 (47)%]	 Loss: 0.561233
Train epoch: 1 [576/1080 (53)%]	 Loss: 0.602836
Train epoch: 1 [640/1080 (59)%]	 Loss: 0.663679
Train epoch: 1 [704/1080 (65)%]	 Loss: 0.792055
Train epoch: 1 [768/1080 (71)%]	 Loss: 0.542309
Train epoch: 1 [832/1080 (76)%]	 Loss: 0.564333
Train epoch: 1 [896/1080 (82)%]	 Loss: 0.765604
Train epoch: 1 [960/1080 (88)%]	 Loss: 0.586473
Train epoch: 1 [896/1080 (94)%]	 Loss: 0.525208
Train epoch: 2 [0/1080 (0)%]	 Loss: 0.553269
Train epoch: 2 [64/1080 (6)%]	 Loss: 0.495875
Train epoch: 2 [128/1080 (12)%]	 Loss: 0.624506
Train epoch: 2 [192/1080 (18)%]	 Loss: 0.526190
Tr

Train epoch: 13 [512/1080 (47)%]	 Loss: 0.414400
Train epoch: 13 [576/1080 (53)%]	 Loss: 0.552185
Train epoch: 13 [640/1080 (59)%]	 Loss: 0.402699
Train epoch: 13 [704/1080 (65)%]	 Loss: 0.652242
Train epoch: 13 [768/1080 (71)%]	 Loss: 0.522769
Train epoch: 13 [832/1080 (76)%]	 Loss: 0.411213
Train epoch: 13 [896/1080 (82)%]	 Loss: 0.413777
Train epoch: 13 [960/1080 (88)%]	 Loss: 0.485226
Train epoch: 13 [896/1080 (94)%]	 Loss: 0.461839
Train epoch: 14 [0/1080 (0)%]	 Loss: 0.403802
Train epoch: 14 [64/1080 (6)%]	 Loss: 0.448780
Train epoch: 14 [128/1080 (12)%]	 Loss: 0.376567
Train epoch: 14 [192/1080 (18)%]	 Loss: 0.419213
Train epoch: 14 [256/1080 (24)%]	 Loss: 0.441956
Train epoch: 14 [320/1080 (29)%]	 Loss: 0.449332
Train epoch: 14 [384/1080 (35)%]	 Loss: 0.490466
Train epoch: 14 [448/1080 (41)%]	 Loss: 0.542483
Train epoch: 14 [512/1080 (47)%]	 Loss: 0.479062
Train epoch: 14 [576/1080 (53)%]	 Loss: 0.498248
Train epoch: 14 [640/1080 (59)%]	 Loss: 0.600549
Train epoch: 14 [704/1080

Train epoch: 23 [832/1080 (76)%]	 Loss: 0.461537
Train epoch: 23 [896/1080 (82)%]	 Loss: 0.435195
Train epoch: 23 [960/1080 (88)%]	 Loss: 0.480457
Train epoch: 23 [896/1080 (94)%]	 Loss: 0.506664
Train epoch: 24 [0/1080 (0)%]	 Loss: 0.471018
Train epoch: 24 [64/1080 (6)%]	 Loss: 0.404012
Train epoch: 24 [128/1080 (12)%]	 Loss: 0.423529
Train epoch: 24 [192/1080 (18)%]	 Loss: 0.626199
Train epoch: 24 [256/1080 (24)%]	 Loss: 0.423190
Train epoch: 24 [320/1080 (29)%]	 Loss: 0.373697
Train epoch: 24 [384/1080 (35)%]	 Loss: 0.458364
Train epoch: 24 [448/1080 (41)%]	 Loss: 0.424422
Train epoch: 24 [512/1080 (47)%]	 Loss: 0.481269
Train epoch: 24 [576/1080 (53)%]	 Loss: 0.405372
Train epoch: 24 [640/1080 (59)%]	 Loss: 0.506563
Train epoch: 24 [704/1080 (65)%]	 Loss: 0.371560
Train epoch: 24 [768/1080 (71)%]	 Loss: 0.452466
Train epoch: 24 [832/1080 (76)%]	 Loss: 0.446726
Train epoch: 24 [896/1080 (82)%]	 Loss: 0.436248
Train epoch: 24 [960/1080 (88)%]	 Loss: 0.484824
Train epoch: 24 [896/1080

Train epoch: 36 [64/1080 (6)%]	 Loss: 0.438152
Train epoch: 36 [128/1080 (12)%]	 Loss: 0.555556
Train epoch: 36 [192/1080 (18)%]	 Loss: 0.376967
Train epoch: 36 [256/1080 (24)%]	 Loss: 0.361452
Train epoch: 36 [320/1080 (29)%]	 Loss: 0.303015
Train epoch: 36 [384/1080 (35)%]	 Loss: 0.444782
Train epoch: 36 [448/1080 (41)%]	 Loss: 0.376462
Train epoch: 36 [512/1080 (47)%]	 Loss: 0.448271
Train epoch: 36 [576/1080 (53)%]	 Loss: 0.473712
Train epoch: 36 [640/1080 (59)%]	 Loss: 0.463227
Train epoch: 36 [704/1080 (65)%]	 Loss: 0.480888
Train epoch: 36 [768/1080 (71)%]	 Loss: 0.413022
Train epoch: 36 [832/1080 (76)%]	 Loss: 0.523134
Train epoch: 36 [896/1080 (82)%]	 Loss: 0.391414
Train epoch: 36 [960/1080 (88)%]	 Loss: 0.365574
Train epoch: 36 [896/1080 (94)%]	 Loss: 0.373657
Train epoch: 37 [0/1080 (0)%]	 Loss: 0.395024
Train epoch: 37 [64/1080 (6)%]	 Loss: 0.289042
Train epoch: 37 [128/1080 (12)%]	 Loss: 0.427081
Train epoch: 37 [192/1080 (18)%]	 Loss: 0.357816
Train epoch: 37 [256/1080 (

Train epoch: 46 [192/1080 (18)%]	 Loss: 0.339384
Train epoch: 46 [256/1080 (24)%]	 Loss: 0.362786
Train epoch: 46 [320/1080 (29)%]	 Loss: 0.343630
Train epoch: 46 [384/1080 (35)%]	 Loss: 0.504295
Train epoch: 46 [448/1080 (41)%]	 Loss: 0.266230
Train epoch: 46 [512/1080 (47)%]	 Loss: 0.310893
Train epoch: 46 [576/1080 (53)%]	 Loss: 0.367136
Train epoch: 46 [640/1080 (59)%]	 Loss: 0.357086
Train epoch: 46 [704/1080 (65)%]	 Loss: 0.365232
Train epoch: 46 [768/1080 (71)%]	 Loss: 0.576287
Train epoch: 46 [832/1080 (76)%]	 Loss: 0.397770
Train epoch: 46 [896/1080 (82)%]	 Loss: 0.244008
Train epoch: 46 [960/1080 (88)%]	 Loss: 0.256602
Train epoch: 46 [896/1080 (94)%]	 Loss: 0.411896
Train epoch: 47 [0/1080 (0)%]	 Loss: 0.335960
Train epoch: 47 [64/1080 (6)%]	 Loss: 0.283025
Train epoch: 47 [128/1080 (12)%]	 Loss: 0.448164
Train epoch: 47 [192/1080 (18)%]	 Loss: 0.436555
Train epoch: 47 [256/1080 (24)%]	 Loss: 0.342660
Train epoch: 47 [320/1080 (29)%]	 Loss: 0.381896
Train epoch: 47 [384/1080

...

Train epoch: 258 [128/1080 (12)%]	 Loss: 0.135142
Train epoch: 258 [192/1080 (18)%]	 Loss: 0.012625
Train epoch: 258 [256/1080 (24)%]	 Loss: 0.063153
Train epoch: 258 [320/1080 (29)%]	 Loss: 0.007682
Train epoch: 258 [384/1080 (35)%]	 Loss: 0.008060
Train epoch: 258 [448/1080 (41)%]	 Loss: 0.058876
Train epoch: 258 [512/1080 (47)%]	 Loss: 0.266534
Train epoch: 258 [576/1080 (53)%]	 Loss: 0.007022
Train epoch: 258 [640/1080 (59)%]	 Loss: 0.054105
Train epoch: 258 [704/1080 (65)%]	 Loss: 0.021367
Train epoch: 258 [768/1080 (71)%]	 Loss: 0.259503
Train epoch: 258 [832/1080 (76)%]	 Loss: 0.087301
Train epoch: 258 [896/1080 (82)%]	 Loss: 0.014966
Train epoch: 258 [960/1080 (88)%]	 Loss: 0.044376
Train epoch: 258 [896/1080 (94)%]	 Loss: 0.187426
Train epoch: 259 [0/1080 (0)%]	 Loss: 0.090482
Train epoch: 259 [64/1080 (6)%]	 Loss: 0.052963
Train epoch: 259 [128/1080 (12)%]	 Loss: 0.159217
Train epoch: 259 [192/1080 (18)%]	 Loss: 0.112500
Train epoch: 259 [256/1080 (24)%]	 Loss: 0.210413
Train

Train epoch: 268 [192/1080 (18)%]	 Loss: 0.004356
Train epoch: 268 [256/1080 (24)%]	 Loss: 0.015774
Train epoch: 268 [320/1080 (29)%]	 Loss: 0.081656
Train epoch: 268 [384/1080 (35)%]	 Loss: 0.061838
Train epoch: 268 [448/1080 (41)%]	 Loss: 0.182689
Train epoch: 268 [512/1080 (47)%]	 Loss: 0.126653
Train epoch: 268 [576/1080 (53)%]	 Loss: 0.017133
Train epoch: 268 [640/1080 (59)%]	 Loss: 0.159497
Train epoch: 268 [704/1080 (65)%]	 Loss: 0.025323
Train epoch: 268 [768/1080 (71)%]	 Loss: 0.013598
Train epoch: 268 [832/1080 (76)%]	 Loss: 0.026663
Train epoch: 268 [896/1080 (82)%]	 Loss: 0.164679
Train epoch: 268 [960/1080 (88)%]	 Loss: 0.007175
Train epoch: 268 [896/1080 (94)%]	 Loss: 0.097001
Train epoch: 269 [0/1080 (0)%]	 Loss: 0.155449
Train epoch: 269 [64/1080 (6)%]	 Loss: 0.067355
Train epoch: 269 [128/1080 (12)%]	 Loss: 0.043749
Train epoch: 269 [192/1080 (18)%]	 Loss: 0.176762
Train epoch: 269 [256/1080 (24)%]	 Loss: 0.038919
Train epoch: 269 [320/1080 (29)%]	 Loss: 0.025278
Train

Train epoch: 278 [256/1080 (24)%]	 Loss: 0.040933
Train epoch: 278 [320/1080 (29)%]	 Loss: 0.021310
Train epoch: 278 [384/1080 (35)%]	 Loss: 0.003114
Train epoch: 278 [448/1080 (41)%]	 Loss: 0.075159
Train epoch: 278 [512/1080 (47)%]	 Loss: 0.038944
Train epoch: 278 [576/1080 (53)%]	 Loss: 0.004765
Train epoch: 278 [640/1080 (59)%]	 Loss: 0.028335
Train epoch: 278 [704/1080 (65)%]	 Loss: 0.024999
Train epoch: 278 [768/1080 (71)%]	 Loss: 0.019274
Train epoch: 278 [832/1080 (76)%]	 Loss: 0.021143
Train epoch: 278 [896/1080 (82)%]	 Loss: 0.011086
Train epoch: 278 [960/1080 (88)%]	 Loss: 0.099780
Train epoch: 278 [896/1080 (94)%]	 Loss: 0.100704
Train epoch: 279 [0/1080 (0)%]	 Loss: 0.182795
Train epoch: 279 [64/1080 (6)%]	 Loss: 0.001385
Train epoch: 279 [128/1080 (12)%]	 Loss: 0.049299
Train epoch: 279 [192/1080 (18)%]	 Loss: 0.079189
Train epoch: 279 [256/1080 (24)%]	 Loss: 0.095744
Train epoch: 279 [320/1080 (29)%]	 Loss: 0.113074
Train epoch: 279 [384/1080 (35)%]	 Loss: 0.006970
Train

Train epoch: 288 [448/1080 (41)%]	 Loss: 0.054541
Train epoch: 288 [512/1080 (47)%]	 Loss: 0.117827
Train epoch: 288 [576/1080 (53)%]	 Loss: 0.013630
Train epoch: 288 [640/1080 (59)%]	 Loss: 0.152971
Train epoch: 288 [704/1080 (65)%]	 Loss: 0.062031
Train epoch: 288 [768/1080 (71)%]	 Loss: 0.048738
Train epoch: 288 [832/1080 (76)%]	 Loss: 0.166311
Train epoch: 288 [896/1080 (82)%]	 Loss: 0.116959
Train epoch: 288 [960/1080 (88)%]	 Loss: 0.080584
Train epoch: 288 [896/1080 (94)%]	 Loss: 0.150977
Train epoch: 289 [0/1080 (0)%]	 Loss: 0.093535
Train epoch: 289 [64/1080 (6)%]	 Loss: 0.035857
Train epoch: 289 [128/1080 (12)%]	 Loss: 0.104791
Train epoch: 289 [192/1080 (18)%]	 Loss: 0.093659
Train epoch: 289 [256/1080 (24)%]	 Loss: 0.135685
Train epoch: 289 [320/1080 (29)%]	 Loss: 0.036333
Train epoch: 289 [384/1080 (35)%]	 Loss: 0.005004
Train epoch: 289 [448/1080 (41)%]	 Loss: 0.017853
Train epoch: 289 [512/1080 (47)%]	 Loss: 0.022803
Train epoch: 289 [576/1080 (53)%]	 Loss: 0.021357
Train

Train epoch: 298 [448/1080 (41)%]	 Loss: 0.011097
Train epoch: 298 [512/1080 (47)%]	 Loss: 0.076304
Train epoch: 298 [576/1080 (53)%]	 Loss: 0.002844
Train epoch: 298 [640/1080 (59)%]	 Loss: 0.017672
Train epoch: 298 [704/1080 (65)%]	 Loss: 0.064183
Train epoch: 298 [768/1080 (71)%]	 Loss: 0.013904
Train epoch: 298 [832/1080 (76)%]	 Loss: 0.080179
Train epoch: 298 [896/1080 (82)%]	 Loss: 0.059378
Train epoch: 298 [960/1080 (88)%]	 Loss: 0.074005
Train epoch: 298 [896/1080 (94)%]	 Loss: 0.004166
Train epoch: 299 [0/1080 (0)%]	 Loss: 0.083198
Train epoch: 299 [64/1080 (6)%]	 Loss: 0.165703
Train epoch: 299 [128/1080 (12)%]	 Loss: 0.079937
Train epoch: 299 [192/1080 (18)%]	 Loss: 0.010781
Train epoch: 299 [256/1080 (24)%]	 Loss: 0.012633
Train epoch: 299 [320/1080 (29)%]	 Loss: 0.112943
Train epoch: 299 [384/1080 (35)%]	 Loss: 0.012706
Train epoch: 299 [448/1080 (41)%]	 Loss: 0.051501
Train epoch: 299 [512/1080 (47)%]	 Loss: 0.051841
Train epoch: 299 [576/1080 (53)%]	 Loss: 0.012709
Train